# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - System instructions
* System instructions are like a preamble that you add before the LLM gets exposed to any further instructions from the user. It lets users steer the behavior of the model based on their specific needs and use cases.
* For example, you can include things like the role or persona, contextual information, and formatting instructions.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instructions



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 14.0 MB/s eta 0:00:00


In [2]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [3]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.



In [4]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [5]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [10]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)


system_instructions = [
 "Use Korean when responding",
 "Use table format to categorize the facts"
]

# Access to the generative model.
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Function to get the response

In [13]:
def generate(query:str):
    """
    Generate a response from the model.

    query : str
      query to be sent to the model

    Returns: str
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [prompt]

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )


    # Print the model response
    print(f'\nUsage metadata:\n{responses.to_dict().get("usage_metadata")}')
    print(f"\nFinish reason:\n{responses.candidates[0].finish_reason}")
    print(f"\nSafety settings:\n{responses.candidates[0].safety_ratings}")


    return responses.text

## Run example

In [14]:

from time import perf_counter

t1_start = perf_counter()

prompt = """
You are a Generative AI specialist to help people understand the concept of AI.
Explain what's the pros and cons of using AI in the following instructions.
1. Define what's the Generative AI.
2. What are differences between typical machine learning and Generative AI.
3. What are the pros and cons of using Generative AI.
4. Explain use cases of Generative AI in the financial industry.

"""

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))



Token Count : total_tokens: 92
total_billable_characters: 321


Usage metadata:
{'prompt_token_count': 103, 'candidates_token_count': 768, 'total_token_count': 871}

Finish reason:
1

Safety settings:
[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.07250525057315826
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.07654563337564468
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.07477004081010818
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.137648344039917
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.14878776669502258
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.08006926625967026
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.046638038009405136
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.04867657274007797
]
Time : 6.897465729999908 seconds




## 생성형 AI 이해를 위한 가이드: 장단점 및 금융 분야 활용 사례

**1. 생성형 AI 정의**

| 용어 | 설명 |
|---|---|
| 생성형 AI | 새로운 데이터를 생성하는 데 특화된 인공지능의 한 유형입니다.  이러한 데이터는 텍스트, 이미지, 오디오, 비디오, 코드와 같은 다양한 형태를 가질 수 있습니다. |

**2. 일반적인 머신 러닝과 생성형 AI의 차이**

|  | 일반적인 머신 러닝 | 생성형 AI | 
|---|---|---|
| 목표 | 기존 데이터를 기반으로 예측 또는 분류 | 새로운 데이터 생성 |
| 사용되는 알고리즘 |  회귀, 분류, 클러스터링 |  생성적 적대 신경망 (GAN), 변형 오토 인코더 (VAE), 확산 모델  |
| 대표적인 예시 |  이미지 분류, 스팸 필터링 |  텍스트 생성, 이미지 생성, 음악 작곡 |

**3. 생성형 AI 사용의 장단점**

|  | 장점 | 단점 | 
|---|---|---|
|  | **새로운 콘텐츠 생성:** 창의적인 콘텐츠를 생성하여 다양한 분야에서 새로운 가능성을 열어줍니다.  | **편향된 결과:** 학습 데이터의 편향을 반영하여 차별적이거나 부정확한 결과를 생성할 수 있습니다.  |
|  | **효율성 증대:** 반복적인 작업을 자동화하여 생산성을 높입니다. | **보안 및 윤리 문제:** 생성된 콘텐츠의 저작권 및 윤리적 문제가 발생할 수 있습니다. |
|  | **개인 맞춤형 경험:** 개인의 취향에 맞는 맞춤형 콘텐츠를 제공합니다. | **데이터 프라이버시:** 개인 정보 보호 및 데이터 유출 위험이 존재합니다. |

**4. 금융 분야에서의 생성형 AI 활용 사례**

|  | 활용 사례 | 설명 |
|---|---|---|
|  | **고객 맞춤형 금융 상품:**  고객의 데이터를 분석하여 개인 맞춤형 금융 상품을 추천하고 설계합니다. | 생성형 AI는 고객의 재무 상황, 소비 패턴, 투자 성향 등을 파악하여 개별 맞춤형 상품을 제공합니다. |
|  | **금융 시장 분석 및 예측:**  금융 시장 데이터를 분석하여 트렌드를 파악하고 미래 시장을 예측합니다. | 생성형 AI는 복잡한 금융 데이터를 처리하고 시장 동향을 분석하여 투자 전략 수립에 도움을 줍니다. |
|  | **사기 탐지 및 예방:**  비정상적인 거래 패턴을 탐지하여 사기 행위를 예방합니다. | 생성형 AI는 이상 거래 패턴을 학습하고 분석하여 사기 시도를 조기에 감지하고 예방합니다. |
|  | **챗봇 및 가상 비서:**  고객 문의에 빠르고 정확하게 응답하여 고객 만족도를 높입니다. | 생성형 AI는 고객 질문에 자연스러운 대화 방식으로 응답하고 필요한 정보를 제공합니다. |
